# Homework 2

## Tasks - Similarity Functions:

1. Which 10 items have the highest Jaccard similarity compared to the first item (i.e., the item from the first review, ‘18471619’)? Report both similarities and item IDs 

In [1]:
import gzip
import math
import numpy as np
import random
from collections import defaultdict 
import matplotlib.pyplot as plt

In [2]:
path="data/goodreads_reviews_comics_graphic.json.gz"

In [3]:
def parseData(path):
  for l in gzip.open(path,'rt',encoding="utf8"):
    yield eval(l)

In [4]:
dataset = list(parseData(path))

In [5]:
dataset[0]

{'user_id': 'dc3763cdb9b2cae805882878eebb6a32',
 'book_id': '18471619',
 'review_id': '66b2ba840f9bd36d6d27f46136fe4772',
 'rating': 3,
 'review_text': 'Sherlock Holmes and the Vampires of London \n Release Date: April 2014 \n Publisher: Darkhorse Comics \n Story by: Sylvain Cordurie \n Art by: Laci \n Colors by: Axel Gonzabo \n Cover by: Jean Sebastien Rossbach \n ISDN: 9781616552664 \n MSRP: $17.99 Hardcover \n "Sherlock Holmes died fighting Professor Moriarty in the Reichenbach Falls. \n At least, that\'s what the press claims. \n However, Holmes is alive and well and taking advantage of his presumed death to travel the globe. \n Unfortunately, Holmes\'s plans are thwarted when a plague of vampirism haunts Britain. \n This book collects Sherlock Holmes and the Vampires of London Volumes 1 and 2, originally created by French publisher Soleil." - Darkhorse Comics \n When I received this copy of "Sherlock Holmes and the Vampires of London" I was Ecstatic! The cover art was awesome and 

In [6]:
usersPerItem=defaultdict(set)
itemsPerUser = defaultdict(set)
ratingDict = {} 

In [7]:
for d in dataset:
    user,item = d['user_id'], d['book_id']
    usersPerItem[item].add(user)
    itemsPerUser[user].add(item)
    ratingDict[(user,item)] = d['rating']

In [8]:
userAverages = {}
itemAverages = {}

for u in itemsPerUser:
    rs = [ratingDict[(u,i)] for i in itemsPerUser[u]]
    userAverages[u] = sum(rs) / len(rs)
    
for i in usersPerItem:
    rs = [ratingDict[(u,i)] for u in usersPerItem[i]]
    itemAverages[i] = sum(rs) / len(rs)

In [9]:
def Jaccard(s1, s2):
    numer = len(s1.intersection(s2))
    denom = len(s1.union(s2))
    if denom == 0:
        return 0
    return numer / denom

In [10]:
def mostSimilar(i):
    similarities = []
    users = usersPerItem[i]
    for i2 in usersPerItem: # For all items
        if i == i2: continue # other than the query
        sim = Jaccard(users, usersPerItem[i2])
        similarities.append((sim,i2))
    similarities.sort(key = lambda x: (-x[0], x[1]),reverse=False)
    return similarities[:10]

In [11]:
query=dataset[0]['book_id']

In [12]:
mostSimilar(query)

[(0.16666666666666666, '25334626'),
 (0.14285714285714285, '25659811'),
 (0.13793103448275862, '18369278'),
 (0.13157894736842105, '18430205'),
 (0.12903225806451613, '20299669'),
 (0.125, '17995154'),
 (0.12121212121212122, '18853527'),
 (0.12121212121212122, '23093378'),
 (0.12121212121212122, '23241671'),
 (0.11764705882352941, '18734070')]

In case of tie, we select the alphabetically first item.

Therefore, the similarities and item IDs are : \
[(0.16666666666666666, '25334626'),
 (0.14285714285714285, '25659811'),
 (0.13793103448275862, '18369278'),
 (0.13157894736842105, '18430205'),
 (0.12903225806451613, '20299669'),
 (0.125, '17995154'),
 (0.12121212121212122, '18853527'),
 (0.12121212121212122, '23093378'),
 (0.12121212121212122, '23241671'),
 (0.11764705882352941, '18734070')]

2. There are several ways similar-item recommendations could be used to make personalized recommenda- tions for a particular user. For instance we could:

(a). Choosing the N items most similar to the user’s favorite (i.e., highest rated) item. 

(b). Finding the N most similar users, and recommending each of their their favorite (highest rated)
items.

Implement these two strategies for user ‘dc3763cdb9b2cae805882878eebb6a32’ (i.e., the user from the first review); in both cases use the Jaccard similarity as your measure of item-to-item or user-to-user similarity. In case of ties, always select the alphabetically first user or item.

(a). Using the first strategy

In [13]:
def getFavoriteId(i,dataset):
    rating=-1
    favoriteId=0
    for d in dataset:
        if d['user_id']==i:
            if d['rating']>rating:
                favoriteId=d['book_id']
                rating=d['rating']
            elif d['rating']==rating and d['book_id']<favoriteId:
                favoriteId=d['book_id']
                rating=d['rating']
    return favoriteId

In [14]:
def mostSimilar(i,N):
    similarities = []
    users = usersPerItem[i]
    for i2 in usersPerItem: # For all items
        if i == i2: continue # other than the query
        sim = Jaccard(users, usersPerItem[i2])
        similarities.append((sim,i2))
    similarities.sort(key = lambda x: (-x[0], x[1]),reverse=False)
    return similarities[:N]

In [15]:
query=getFavoriteId('dc3763cdb9b2cae805882878eebb6a32',dataset)

In [16]:
mostSimilar(query,10)

[(0.16666666666666666, '25334626'),
 (0.14285714285714285, '25659811'),
 (0.13793103448275862, '18369278'),
 (0.13157894736842105, '18430205'),
 (0.12903225806451613, '20299669'),
 (0.125, '17995154'),
 (0.12121212121212122, '18853527'),
 (0.12121212121212122, '23093378'),
 (0.12121212121212122, '23241671'),
 (0.11764705882352941, '18734070')]

In case of tie, we select the alphabetically first item. \
In the first Strategy, the recommended item IDs and their scores are : \
[(0.16666666666666666, '25334626'),
 (0.14285714285714285, '25659811'),
 (0.13793103448275862, '18369278'),
 (0.13157894736842105, '18430205'),
 (0.12903225806451613, '20299669'),
 (0.125, '17995154'),
 (0.12121212121212122, '18853527'),
 (0.12121212121212122, '23093378'),
 (0.12121212121212122, '23241671'),
 (0.11764705882352941, '18734070')].

(b). Using the second strategy

There we should note that if the similarity between two users is 1, then they consumed exactly same items, we do not add it to N most similar users. Also if the user has already consumed the favorite item of a similar user, we move to the next similar user.

In [17]:
def mostSimilar(i):
    similarities = []
    items = itemsPerUser[i]
    for i2 in itemsPerUser: # For all items
        if i == i2: continue # other than the query
        sim = Jaccard(items, itemsPerUser[i2])
        if sim != 1:
            similarities.append((sim,i2))
    similarities.sort(key = lambda x: (-x[0], x[1]),reverse=False)
    return similarities

In [18]:
def getFavoriteinSimilarities(i,N):
    cnt=0
    res=[]
    for s in similarities:
        favoriteItem=getFavoriteId(s[1],dataset)
        if favoriteItem in itemsPerUser[i]:
            continue
        res.append((s[0],favoriteItem))
        cnt+=1
        if cnt==N:
            break
    return res

In [19]:
query='dc3763cdb9b2cae805882878eebb6a32'

In [20]:
similarities=mostSimilar(query)

In [21]:
recommending=getFavoriteinSimilarities(query,10)
recommending

[(0.3333333333333333, '10767466'),
 (0.25, '17570797'),
 (0.2, '15704307'),
 (0.14285714285714285, '10138607'),
 (0.05555555555555555, '12434747'),
 (0.030303030303030304, '17995248'),
 (0.023809523809523808, '10105459'),
 (0.02040816326530612, '10997645'),
 (0.014925373134328358, '10361139'),
 (0.0136986301369863, '10264328')]

In case of ties, always select the alphabetically first user or item. \
In the second Strategy, the scores and recommended item IDs are : \
[(0.3333333333333333, '10767466'),
 (0.25, '17570797'),
 (0.2, '15704307'),
 (0.14285714285714285, '10138607'),
 (0.05555555555555555, '12434747'),
 (0.030303030303030304, '17995248'),
 (0.023809523809523808, '10105459'),
 (0.02040816326530612, '10997645'),
 (0.014925373134328358, '10361139'),
 (0.0136986301369863, '10264328')]

3. In class we briefly discussed whether the Pearson similarity should be implemented (a) only in terms of shared items (i.e., $U_i ∩ U_j$ ) in the denominator; or (b) in terms of all items each user consumed (i.e., $U_i$ or $U_j$ for each term in the denominator). (See last slide on Pearson similarity). Implement versions of the Pearson similarity based on both definitions, and report the 10 most similar items to the same query item from Question 1 

(a) only in terms of shared items in the denominator

In [22]:
def Pearson(i1, i2):
    iBar1 = itemAverages[i1]
    iBar2 = itemAverages[i2]
    inter = usersPerItem[i1].intersection(usersPerItem[i2])
    numer = 0 
    denom1 = 0
    denom2 = 0
    for u in inter:
        numer += (ratingDict[(u,i1)] - iBar1)*(ratingDict[(u,i2)] - iBar2)
    for u in inter: 
        denom1 += (ratingDict[(u,i1)] - iBar1)**2
        denom2 += (ratingDict[(u,i2)] - iBar2)**2
    denom = math.sqrt(denom1) * math.sqrt(denom2)
    if denom == 0: return 0
    return numer / denom

In [23]:
def mostSimilar(i):
    similarities = []
    users = usersPerItem[i]
    for i2 in usersPerItem: 
        if i == i2: continue 
        sim = Pearson(i, i2)
        similarities.append((sim,i2))
    similarities.sort(key = lambda x: (-x[0], x[1]),reverse=False)
    return similarities[:10]

In [24]:
query=dataset[0]['book_id']

In [25]:
mostSimilar(query)

[(1.0000000000000002, '1103951'),
 (1.0000000000000002, '11986350'),
 (1.0000000000000002, '16007365'),
 (1.0000000000000002, '17132269'),
 (1.0000000000000002, '17571519'),
 (1.0000000000000002, '18468852'),
 (1.0000000000000002, '18527488'),
 (1.0000000000000002, '18594657'),
 (1.0000000000000002, '18624024'),
 (1.0000000000000002, '1882498')]

In case of ties, always select the alphabetically first item.
Therefore, the similarities and item IDs are : \
[(1.0000000000000002, '1103951'),
 (1.0000000000000002, '11986350'),
 (1.0000000000000002, '16007365'),
 (1.0000000000000002, '17132269'),
 (1.0000000000000002, '17571519'),
 (1.0000000000000002, '18468852'),
 (1.0000000000000002, '18527488'),
 (1.0000000000000002, '18594657'),
 (1.0000000000000002, '18624024'),
 (1.0000000000000002, '1882498')]

(b) in terms of all items each user consumed in the denominator

In [26]:
def Pearson(i1, i2):
    # Between two items
    iBar1 = itemAverages[i1]
    iBar2 = itemAverages[i2]
    inter = usersPerItem[i1].intersection(usersPerItem[i2])
    numer = 0
    denom1 = 0
    denom2 = 0
    for u in inter:
        numer += (ratingDict[(u,i1)] - iBar1)*(ratingDict[(u,i2)] - iBar2)
    for u in usersPerItem[i1]:
        denom1 += (ratingDict[(u,i1)] - iBar1)**2
    for u in usersPerItem[i2]:
        denom2 += (ratingDict[(u,i2)] - iBar2)**2
    denom = math.sqrt(denom1) * math.sqrt(denom2)
    if denom == 0: return 0
    return numer / denom

In [27]:
def mostSimilar(i):
    similarities = []
    users = usersPerItem[i]
    for i2 in usersPerItem: # For all items
        if i == i2: continue # other than the query
        sim = Pearson(i, i2)
        similarities.append((sim,i2))
    similarities.sort(key = lambda x: (-x[0], x[1]),reverse=False)
    return similarities[:10]

In [28]:
query=dataset[0]['book_id']

In [29]:
mostSimilar(query)

[(0.31898549007874194, '20300526'),
 (0.18785865431369264, '13280885'),
 (0.17896391275176457, '18208501'),
 (0.16269036695641687, '21521612'),
 (0.16269036695641687, '25430791'),
 (0.1555075595594449, '1341758'),
 (0.1526351566298752, '6314737'),
 (0.15204888048160353, '4009034'),
 (0.1494406444160154, '988744'),
 (0.14632419481281994, '18430205')]

Therefore, the similarities and item IDs are : \
[(0.31898549007874194, '20300526'),
 (0.18785865431369264, '13280885'),
 (0.17896391275176457, '18208501'),
 (0.16269036695641687, '21521612'),
 (0.16269036695641687, '25430791'),
 (0.1555075595594449, '1341758'),
 (0.1526351566298752, '6314737'),
 (0.15204888048160353, '4009034'),
 (0.1494406444160154, '988744'),
 (0.14632419481281997, '18430205')]

## Tasks - Rating Prediction

4. Implement a rating prediction model based on the similarity function

In [30]:
reviewsPerUser = defaultdict(list)
reviewsPerItem = defaultdict(list)

In [31]:
for d in dataset:
    user,item = d['user_id'], d['book_id']
    reviewsPerUser[user].append(d)
    reviewsPerItem[item].append(d)

In [32]:
def predictRating(user,item):
    ratings = []
    similarities = []
    for d in reviewsPerUser[user]:
        i2 = d['book_id']
        if i2 == item: continue
        ratings.append(d['rating'] - itemAverages[i2])
        similarities.append(Jaccard(usersPerItem[item],usersPerItem[i2]))
    if (sum(similarities) > 0):
        weightedRatings = [(x*y) for x,y in zip(ratings,similarities)]
        return itemAverages[item] + sum(weightedRatings) / sum(similarities)
    else:
        # User hasn't rated any similar items
        return itemAverages[item]

To save time, we use the first 10000 samples of the dataset to calculate MSE.

In [33]:
subset=dataset[:10000]

In [34]:
rating_estimate=[predictRating(d['user_id'], d['book_id']) for d in subset]

In [35]:
rating=[d['rating'] for d in subset]

In [36]:
MSE=(np.square(np.subtract(rating, rating_estimate))).mean()
MSE

0.7017041185560418

6. Design a decay function that outperforms (in terms of the MSE) the trivial function f(tu,j) = 1, documenting any design choices you make. 

In [37]:
dataset[0]

{'user_id': 'dc3763cdb9b2cae805882878eebb6a32',
 'book_id': '18471619',
 'review_id': '66b2ba840f9bd36d6d27f46136fe4772',
 'rating': 3,
 'review_text': 'Sherlock Holmes and the Vampires of London \n Release Date: April 2014 \n Publisher: Darkhorse Comics \n Story by: Sylvain Cordurie \n Art by: Laci \n Colors by: Axel Gonzabo \n Cover by: Jean Sebastien Rossbach \n ISDN: 9781616552664 \n MSRP: $17.99 Hardcover \n "Sherlock Holmes died fighting Professor Moriarty in the Reichenbach Falls. \n At least, that\'s what the press claims. \n However, Holmes is alive and well and taking advantage of his presumed death to travel the globe. \n Unfortunately, Holmes\'s plans are thwarted when a plague of vampirism haunts Britain. \n This book collects Sherlock Holmes and the Vampires of London Volumes 1 and 2, originally created by French publisher Soleil." - Darkhorse Comics \n When I received this copy of "Sherlock Holmes and the Vampires of London" I was Ecstatic! The cover art was awesome and 

In [38]:
import dateutil.parser

In [39]:
dateutil.parser.parse(dataset[0]['date_added']).timestamp()

1386269065.0

$$
r(u,i)=\bar{R_i}+\frac{\sum_{j\in I_u}(R_{u,j}-\bar{R_j})\cdot Sim(i,j) \cdot f(t_{u,j}) }{\sum_{j \in I_u}Sim(i,j)\cdot f(t_{u,j})}
$$

We use the property 'date_added' as the $t_{u,j}$ here, and use the folowing decay function:
$$
f(t)=e^{-\lambda t}
$$
 We first use the decay form $f(|t_{u,i}-t_{u,j}|)$

In [40]:
def predictRating(user,item, a):
    ratings = []
    similarities = []
    decay= []
    time1=0.0
    for d in reviewsPerUser[user]:
        i2=d['book_id']
        if i2==item:
            time1=dateutil.parser.parse(d['date_added']).timestamp()
            break
    for d in reviewsPerUser[user]:
        i2 = d['book_id']
        if i2 == item: continue
        ratings.append(d['rating'] - itemAverages[i2])
        time2=dateutil.parser.parse(d['date_added']).timestamp()
        similarities.append(Jaccard(usersPerItem[item],usersPerItem[i2]))
        decay.append(math.exp(-a*abs(time1-time2)))
    weightedSimilarities=[(y*z) for y,z in zip(similarities, decay)]
    if (sum(weightedSimilarities) > 0):
        weightedRatings = [(x*y*z) for x,y,z in zip(ratings,similarities, decay)]
        return itemAverages[item] + sum(weightedRatings) / sum(weightedSimilarities)
    else:
        # User hasn't rated any similar items
        return itemAverages[item]

According to the fact that the timestamp has 10 digits, we try $10^{-10}、10^{-9}、10{-8}$

In [41]:
rating_estimate=[predictRating(d['user_id'], d['book_id'],0.0000000001) for d in subset]
MSE=(np.square(np.subtract(rating, rating_estimate))).mean()
MSE

0.7016455656552574

In [42]:
rating_estimate=[predictRating(d['user_id'], d['book_id'],0.0000000001*10) for d in subset]
MSE=(np.square(np.subtract(rating, rating_estimate))).mean()
MSE

0.701143620482529

In [43]:
rating_estimate=[predictRating(d['user_id'], d['book_id'],0.0000000001*100) for d in subset]
MSE=(np.square(np.subtract(rating, rating_estimate))).mean()
MSE

0.6979483111146024

Therefore, we use the MSE when we set $\lambda=10^{-8}$, and compared to the origin MSE when $f(tu,j) = 1$, the improvement is as below:

In [44]:
0.7017041185560418/0.6979483111146024-1

0.005381211447365475

which is 0.5%

So my decay function is:
$$
f(|t_{u,i}-t_{u,j}|)=e^{-10^{-8} |t_{u,i}-t_{u,j}|}
$$